# 06_refinement_silver_cleanup_job
---
Este job remove arquivos intermediários da camada **Silver**, mantendo apenas o arquivo final `flights_aggregated.parquet` na partição.

In [10]:
run_mode = "latest"
run_date = None

silver_path = "/opt/airflow/data-layer/silver"


In [11]:
from pathlib import Path
from transformer.utils.spark_helpers import get_spark_session
from transformer.utils.file_io import find_partition
from transformer.utils.logger import get_logger
import shutil

log = get_logger("refinement.silver_cleanup")

spark = get_spark_session("SilverCleanupJob")
log.info("[Refinement][Aggregate] Sessão Spark iniciada.")

# Ajustes performance
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "32")


2025-11-13 23:51:56 [INFO] refinement.silver_cleanup | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-13 23:51:56 [INFO] spark_helpers | [INFO] SparkSession criada com sucesso: 'SilverCleanupJob' (master=local[*]).
2025-11-13 23:51:56 [INFO] refinement.silver_cleanup | [Refinement][Aggregate] Sessão Spark iniciada.


In [12]:
try:
    log.info("[Refinement][SilverCleanup] Iniciando job de limpeza da camada silver.")

    partition = find_partition(
        base_path=silver_path,
        mode=run_mode,
        date_str=run_date,
    )
    partition_dir = Path(silver_path) / partition / "PARQUET"

    if not partition_dir.exists():
        raise FileNotFoundError(
            f"[Refinement][SilverCleanup][ERROR] Diretório de partição não encontrado: {partition_dir}."
        )

    # Apenas este diretório deve permanecer
    keep = {"flights_aggregated.parquet"}

    log.info(f"[Refinement][SilverCleanup] Diretório alvo: {partition_dir}.")
    log.info(f"[Refinement][SilverCleanup] Mantendo apenas: {keep}.")

    for item in partition_dir.iterdir():
        # Se for o agregado, não mexe
        if item.name in keep:
            log.info(f"[Refinement][SilverCleanup] Mantendo: {item}")
            continue

        # Remove diretórios intermediários
        if item.is_dir():
            try:
                shutil.rmtree(item)
                log.info(f"[Refinement][SilverCleanup] Diretório removido: {item}.")
            except Exception as e:
                log.warning(f"[Refinement][SilverCleanup] Falha ao remover diretório {item}: {e}.")
            continue

        # Remove arquivos soltos (se houver)
        if item.is_file():
            try:
                item.unlink()
                log.info(f"[Refinement][SilverCleanup] Arquivo removido: {item}.")
            except Exception as e:
                log.warning(f"[Refinement][SilverCleanup] Falha ao remover arquivo {item}: {e}.")

    log.info("[Refinement][SilverCleanup] Limpeza concluída com sucesso.")

except Exception as e:
    log.exception(f"[Refinement][SilverCleanup][ERROR] Falha durante o cleanup: {e}")
    raise

finally:
    log.info("[Refinement][SilverCleanup] Job de limpeza da camada silver encerrado.")


2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refinement][SilverCleanup] Iniciando job de limpeza da camada silver.
2025-11-13 23:51:57 [INFO] file_io | [INFO] Partição selecionada: 2025-11-12
2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refinement][SilverCleanup] Diretório alvo: /opt/airflow/data-layer/silver/2025-11-12/PARQUET.
2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refinement][SilverCleanup] Mantendo apenas: {'flights_aggregated.parquet'}.
2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refinement][SilverCleanup] Diretório removido: /opt/airflow/data-layer/silver/2025-11-12/PARQUET/airport.parquet.
2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refinement][SilverCleanup] Mantendo: /opt/airflow/data-layer/silver/2025-11-12/PARQUET/flights_aggregated.parquet
2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refinement][SilverCleanup] Limpeza concluída com sucesso.
2025-11-13 23:51:57 [INFO] refinement.silver_cleanup | [Refin

In [15]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

partition = find_partition(silver_path, mode=run_mode, date_str=run_date)
for item in (Path(silver_path) / partition / "PARQUET").iterdir():
    print(item)


2025-11-13 23:52:15 [INFO] file_io | [INFO] Partição selecionada: 2025-11-12


/opt/airflow/data-layer/silver/2025-11-12/PARQUET/flights_aggregated.parquet


In [16]:
# Encerra a sessão Spark
spark.stop()
log.info("[Refinement][Aggregate] Sessão Spark finalizada.")


2025-11-13 23:52:23 [INFO] refinement.silver_cleanup | [Refinement][Aggregate] Sessão Spark finalizada.
